# ETL

## IMPORTANDO LIBRERIAS 

In [20]:

import pandas as pd
import numpy as np
# Módulo para trabajar con rutas de archivo
from pathlib import Path
# Librería para identificar encondings
import chardet
#importamos lalibreria os para trabajar con los archivos
import os


## cargando data set

In [21]:
#funcion para leer archivos csv y json a partir de una ruta
def get_data(filepath):
    #manejando los archivos csv y json
    with open(filepath, 'rb') as file:
        result = chardet.detect(file.read())
        # Para importar archivos csv, detectando encoding
        if Path(filepath).suffix == ".csv": 
            DF = pd.read_csv(filepath, encoding=result['encoding'])
        # Para importar archivos json
        elif Path(filepath).suffix == ".json": 
            DF = pd.read_json(filepath) 

    return DF

#en esta funcion podemos añadir los archivos que queremos leer segun nuestras necesidades

In [22]:
#funcionque nos da el nombres de los archivos a partir de la ruta del   archivo
def get_nombre_archivo(filepath):
    #leemos el contenido de la ruta 
    archivos = os.listdir(filepath)
    archivos_sin_extension = [os.path.splitext(nombre_archivo)[0] for nombre_archivo in archivos if os.path.splitext(nombre_archivo)[1] in ['.csv','.json'] ]
    return archivos_sin_extension

In [23]:
#creando  los data framees a partir de los nombres que nos dieron
i=1
directorio='./data/'

for nombre_archivo in get_nombre_archivo(directorio):
    locals()[nombre_archivo]=get_data(f'{directorio}{nombre_archivo}.csv')
    print(f'el nombre de tu variable {i} es :',nombre_archivo)
    i+=1

el nombre de tu variable 1 es : disney_plus_titles
el nombre de tu variable 2 es : amazon_prime_titles
el nombre de tu variable 3 es : netflix_titles
el nombre de tu variable 4 es : hulu_titles


In [24]:

#bucle para ver el esquema de las df 
for df in get_nombre_archivo(directorio):
    print('-'*50)
    print(f'las caractericas del df {df} es :',)
    print('-'*50)
    print(locals()[df].info())

--------------------------------------------------
las caractericas del df disney_plus_titles es :
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450 entries, 0 to 1449
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       1450 non-null   object
 1   type          1450 non-null   object
 2   title         1450 non-null   object
 3   director      977 non-null    object
 4   cast          1260 non-null   object
 5   country       1231 non-null   object
 6   date_added    1447 non-null   object
 7   release_year  1450 non-null   int64 
 8   rating        1447 non-null   object
 9   duration      1450 non-null   object
 10  listed_in     1450 non-null   object
 11  description   1450 non-null   object
dtypes: int64(1), object(11)
memory usage: 136.1+ KB
None
--------------------------------------------------
las caractericas del df amazon_prime_titles es

In [25]:
amazon_prime_titles.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       9668 non-null   object
 1   type          9668 non-null   object
 2   title         9668 non-null   object
 3   director      7585 non-null   object
 4   cast          8435 non-null   object
 5   country       672 non-null    object
 6   date_added    155 non-null    object
 7   release_year  9668 non-null   int64 
 8   rating        9331 non-null   object
 9   duration      9668 non-null   object
 10  listed_in     9668 non-null   object
 11  description   9668 non-null   object
dtypes: int64(1), object(11)
memory usage: 906.5+ KB


### Inputacion de datos

In [26]:
for df in get_nombre_archivo(directorio):
    locals()[df].set_index('title', drop=True, inplace=True)

In [27]:
for df in get_nombre_archivo(directorio):
    for data in get_nombre_archivo(directorio):
        if data != df:
            locals()[df].fillna(locals()[data], inplace=True)

In [28]:
for df in get_nombre_archivo(directorio):
    locals()[df].reset_index(inplace=True)

## Transformaciones

In [29]:
# Generando el campo id
for df in get_nombre_archivo(directorio):
    locals()[df]['id'] = df[0]+locals()[df]['show_id']

In [30]:
#Remplazando los valores nulos por G
for df in get_nombre_archivo(directorio):
    locals()[df]['rating'].fillna('G', inplace=True)
    locals()[df]['rating'].astype(str)

In [31]:

#dando  formato AAAA-mm-dd a las fechas 
for df in get_nombre_archivo(directorio):
    #formato de las fechasoriginales 
    date_format ='%B %d, %Y'
    #convertimos las fechas a formato AAAA-mm-dd
    locals()[df]['date_added']=pd.to_datetime(locals()[df]['date_added'].str.strip(),format=date_format)
    #con esta linea de codigo podemos darle el formato que queramos a las fechas
    #locals()[df]['date_added'] = locals()[df]['date_added'].strftime("%d %B %Y")



In [32]:
#Los campos de texto deberán estar en minúsculas, sin excepciones
for df in get_nombre_archivo(directorio):
    colunas_obj=locals()[df].select_dtypes('object').columns
    locals()[df][colunas_obj]=(
                                locals()[df][colunas_obj]
                               .apply(lambda x: x.str.lower())
                               )

In [33]:
#El campo duration debe convertirse en dos campos: duration_int y duration_unit

for df in get_nombre_archivo(directorio):
    locals()[df]['duration'].fillna('sin dato', inplace=True)
    locals()[df]['duration_int'] = locals()[df]['duration'].apply(lambda x: x.split(' ')[0])
    locals()[df]['duration_int'].replace('sin','0', inplace=True)
    locals()[df]['duration_int']=locals()[df]['duration_int'].astype(int)
    locals()[df]['duration_unit'] = locals()[df]['duration'].apply(lambda x: x.split(' ')[1])

# Exportando los datos 

In [42]:
get_nombre_archivo(directorio)

['disney_plus_titles', 'amazon_prime_titles', 'netflix_titles', 'hulu_titles']

In [44]:
'disney_plus_titles'.split('_')[0]

'disney'

In [45]:
#unir todas las pelicula
movies=pd.DataFrame()
for df in get_nombre_archivo(directorio):
    locals()[df]['plataforma']=df.split('_')[0]
    movies=pd.concat([movies,locals()[df]],ignore_index=True)


In [46]:
del movies['duration']
del movies['show_id']

In [48]:
#exportando los datos a un archivo csv
movies.to_csv('./data/data_procesada/movie.csv', sep=',', index=False, encoding='utf-8')

In [50]:
movies=pd.read_csv('./data/data_procesada/movie.csv',sep=',',encoding='utf-8',index_col='id')
movies.head(5)

,title,type,director,cast,country,date_added,release_year,rating,listed_in,description,duration_int,duration_unit,plataforma
id,,,,,,,,,,,,,
ds1,duck the halls: a mickey mouse christmas special,movie,"alonso ramirez ramos, dave wasson","chris diamantopoulos, tony anselmo, tress macn...",NaN,2021-11-26,2016,tv-g,"animation, family",join mickey and the gang as they duck the halls!,23,min,disney
ds2,ernest saves christmas,movie,john cherry,"jim varney, noelle parker, douglas seale",NaN,2021-11-26,1988,pg,comedy,santa claus passes his magic bag to a new st. ...,91,min,disney
ds3,ice age: a mammoth christmas,movie,karen disher,"raymond albert romano, john leguizamo, denis l...",united states,2021-11-26,2011,tv-g,"animation, comedy, family",sid the sloth is on santa's naughty list.,23,min,disney
ds4,the queen family singalong,movie,hamish hamilton,"darren criss, adam lambert, derek hough, alexa...",NaN,2021-11-26,2021,tv-pg,musical,"this is real life, not just fantasy!",41,min,disney
ds5,the beatles: get back,tv show,NaN,"john lennon, paul mccartney, george harrison, ...",NaN,2021-11-25,2021,g,"docuseries, historical, music",a three-part documentary from peter jackson ca...,1,season,disney


In [53]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22998 entries, ds1 to hs3073
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          22998 non-null  object
 1   type           22998 non-null  object
 2   director       15057 non-null  object
 3   cast           18123 non-null  object
 4   country        12078 non-null  object
 5   date_added     14017 non-null  object
 6   release_year   22998 non-null  int64 
 7   rating         22998 non-null  object
 8   listed_in      22998 non-null  object
 9   description    22994 non-null  object
 10  duration_int   22998 non-null  int64 
 11  duration_unit  22998 non-null  object
 12  plataforma     22998 non-null  object
dtypes: int64(2), object(11)
memory usage: 3.0+ MB


In [38]:
movies.columns

Index(['title', 'type', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'listed_in', 'description', 'duration_int',
       'duration_unit'],
      dtype='object')